In [1]:
#author: Irsyad Adam

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


## Loading Cora Dataset

In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Extracting file to C:\Users\irsya\.dgl\cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [10]:
graph = dataset[0]
print(graph)

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(1433,), dtype=torch.float32)}
      edata_schemes={})


## Building Multi - Layer GCN

Each layer computes new node representations by aggregating neighbor information

To build a multi-layer GCN you can simply stack dgl.nn.GraphConv modules, which inherit torch.nn.Module()

In [15]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats: list, h_feats: list, num_classes: int) -> None:
        """
        @param in_feats is the input
        @param h_feats is ???
        @param num_classes is the number of classes that the model 
        """
        #create the initialization of the GCN
        super(GCN, self).__init__()

        #2 layers, graph conv
        #first layer is the basic layer, takes in fteauts, spits out h features
        self.conv1 = GraphConv(in_feats, h_feats)
        #second layer takes in the output of first and spits out num of classes
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feats: int):
        #gets called every instance of the class
        h = self.conv1(g, in_feats)
        f = F.relu(h)
        h = self.conv2
        return h

#creat the model
model = GCN(graph.ndata["feat"].shape[1], 16, dataset.num_classes)

In [19]:
print("num classes:",  dataset.num_classes)
print(graph.ndata["feat"].shape[1])

num classes: 7
1433


In [ ]:
def train(g, model):
    